In [ ]:
%reset -f

In [ ]:
import os
import numpy as np
from scipy import interpolate
import scipy.io
import Rbeast as rb
from SLM_tools import *
from scipy.io import savemat
import pickle
import re
import time

In [ ]:
top_dir = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\Final Project\Project2-Omri and Idan\Results" 

pattern = re.compile(
    r"(?P<type>distance|energy)_vec_mu_(?P<mu>[-+]?\d*\.\d+|\d+)_energy_(?P<energy>[-+]?\d*\.\d+|\d+)_run_num_(?P<run_num>\d+)_total_num_target_\d+\.mat"
)

distance_paths = []
energy_paths = []
seen_distance_paths = set()
seen_energy_paths = set()
mu = 1.6
for root, dirs, files in os.walk(top_dir):
    for file in files:
        match = pattern.match(file)
        if match:
            file_info = match.groupdict()
            if file_info['mu'] == '1.6':
                full_path = os.path.join(root, file)
                if file_info['type'] == 'distance':
                    if full_path not in seen_distance_paths:
                        distance_paths.append(full_path)
                        seen_distance_paths.add(full_path)
                elif file_info['type'] == 'energy':
                    if full_path not in seen_energy_paths:
                        energy_paths.append(full_path)
                        seen_energy_paths.add(full_path)


In [ ]:
len(energy_paths)

In [ ]:
len(distance_paths)

In [ ]:
C = []
downsampling_factor = 1000
n_components = 3
features_extracted = 0
total_runtime = 0
files_with_assembly = 0
total_data_files = len(distance_paths)
log = []
mu = 1.6
counters = {"load":[],"extraction":[],"segmentation":[],"pca":None,"extracted_features":[]}
for i, (energy_path, distance_path) in enumerate(zip(energy_paths, distance_paths)):
    start1 = time.time()
    distance_data = scipy.io.loadmat(distance_path)['foo']
    energy_data = scipy.io.loadmat(energy_path)['foo'].T
    energy_data, distance_data, time_vec = SLM_tools.downsample(energy_data, distance_data,downsampling_factor)
    loading_time = round(time.time() - start1,3)
    log1 = f"\nfile {i+1}/{len(energy_paths)}  loaded: {os.path.relpath(energy_path,top_dir)} Loading time: {loading_time} seconds"
    print(log1)
    log.append(log1)
    counters["load"].append(loading_time)
    start2 = time.time()
    o, cp, mean_trend = SLM_tools.beast(energy_data)
    segmentation_time = round(time.time() - start2,3)
    log2 = f"\nfinished segmentation(Beast), runtime: {segmentation_time} seconds"
    print(log2)
    log.append(log2)
    counters["segmentation"].append(segmentation_time)
    start3 = time.time()
    A = SLM_tools.segment_data(energy_data, distance_data,mean_trend,cp)
    extraction_time = round(time.time() - start3,3)
    print(f"\nfinished feature extraction, runtime: {extraction_time} seconds")
    counters["extraction"].append(extraction_time)
    print(f"\nAssembly? : {len(A) > 0}")
    if len(A) > 0:
        files_with_assembly += 1
        features_extracted += A[0].shape[0]
        counters["extracted_features"].append(A[0].shape[0])
        log3 = f"\nfiles_with_assembly: {files_with_assembly}/{total_data_files}"
        print(log3)
        log.append(log3)
        log4 = f"\nTotal features_extracted: {features_extracted}"
        print(log4)
        log.append(log4)
    else:
        log5 = f"\nNo Assembly, No features extracted"
        print(log5)
        log.append(log5)
    C.append(A)
    total_runtime += (loading_time + extraction_time + segmentation_time)
log6 = f"\nFinished pre-pca processing,total runtime {total_runtime} for {len(energy_paths)} files"
print(log6)
log.append(log6)
with open(f"C_all_mu_{mu}.pkl", "wb") as f:
    pickle.dump(C, f)
log7 = "\nsaved all processed data"
print(log7)
log.append(log7)
start_pca = time.time()
log8 = f"\nStarting PCA"
print(log8)
log.append(log8)
c_reduced = SLM_tools.post_beast_processing(C)
principal_components, score, latent = SLM_tools.pca(c_reduced,n_components)
a_reduced = SLM_tools.post_pca_processing(score, c_reduced,n_components)
runtime_pca = round(time.time() - start_pca,3)
counters["pca"] = runtime_pca
total_runtime += runtime_pca
log9 = f"\nFinished PCA runtime {runtime_pca} seconds"
print(log9)
log.append(log9)
savemat(f"a_reduced_all_mu_{mu}.mat", {'a_reduced': a_reduced})
log10 = f"\nsaved all extracted features , total extracted features {features_extracted} total runtime {total_runtime} seconds"
log.append(log10)
log.append("SUMMARY")
log11 = f"\nfiles {total_data_files} \nfiles_with_assembly: {files_with_assembly}/{total_data_files} \nmean extracted features {np.mean(counters['extracted_features'])} \ntotal extracted features {features_extracted} \ntotal runtime {total_runtime} seconds \nmean loading time {np.mean(counters['load'])} \nmean segmentation time {np.mean(counters['segmentation'])} \nmean extraction time {np.mean(counters['extraction'])} \nPCA time {counters['pca']}"
log.append(log11)
print(log11)
with open(f"run_log_27_7_mu_{mu}.txt",'w') as f:
    for line in log:
        f.write(line)

In [ ]:
import os
import numpy as np
from scipy import interpolate
import scipy.io
import Rbeast as rb
from SLM_tools import *
from scipy.io import savemat
import pickle
import re
import time
%matplotlib inline

In [ ]:
a_reduced = sio.loadmat(r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\not_used\a_reduced_all_mu_0.6_2.mat")["a_reduced"]

In [ ]:
a_reduced.shape

In [ ]:
path=r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\not_used"
cv_num = 10

In [ ]:
YI, tfas_predict_mat, tfas_actually_mat, mean_error_mat, train_index, random_x, validation_index, median_fit_vec = SLM_tools.model_training_with_cv(a_reduced,n_components=3,cv_num=10)

In [ ]:
mean_vec, std_vec, hist_space, x_hist_space, fig_1, a, b, x_ticks, y_ticks, bin_width= SLM_tools.model_eval(tfas_predict_mat, tfas_actually_mat, cv_num, path)

In [ ]:
tfas_predict_mat_2, tfas_actually_mat_2, mean_error_mat_2 = SLM_tools.train_again_on_validation_and_test(random_x, validation_index, n_components=3)

In [ ]:
SLM_tools.cv_bias_correction(tfas_predict_mat_2, tfas_actually_mat_2, hist_space,bin_width, mean_vec, x_hist_space,
                           median_fit_vec, x_ticks, y_ticks, path)

In [ ]:
707%10